# HuggingFace로 두 문장의 논리적 모순 분류하기

이번 실습에서는 HuggingFace로 영화리뷰 감정 분석과 같은 text 분류 문제를 위한 모델을 구현합니다.
먼저 필요한 library들을 설치하고 import합니다.

In [18]:
!pip install transformers datasets evaluate accelerate scikit-learn openai==0.28


  Attempting uninstall: openai
    Found existing installation: openai 1.59.3
    Uninstalling openai-1.59.3:
      Successfully uninstalled openai-1.59.3


In [7]:
!ls /content

!git clone https://github.com/NomaDamas/KICE_slayer_AI_Korean.git

!ls ./KICE_slayer_AI_Korean/data

ls: /content: No such file or directory
fatal: destination path 'KICE_slayer_AI_Korean' already exists and is not an empty directory.
2023_11_KICE.json


## [MY CODE] 2023_11_KICE.json 데이터셋 로드

2023년도 해당 수능문제를 로드합니다.

In [3]:
import json

# 데이터 로드
file_path = './KICE_slayer_AI_Korean/data/2023_11_KICE.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 데이터 구조 확인
print(type(data))  # 리스트인지 확인
print(len(data))   # 데이터 길이

# 첫 번째 문제 출력
example_problem = data[0]  # 리스트에서 첫 번째 문제 추출
print(json.dumps(example_problem, indent=2, ensure_ascii=False))


<class 'list'>
11
{
  "id": "2023_11_KICE_1-3",
  "paragraph": "사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 

In [4]:
# 'problems'에서 첫 번째 문제 추출
first_problem = example_problem['problems'][0]  # 첫 번째 문제 선택

# 문제와 선택지 출력
question = first_problem['question']  # 문제 내용
choices = first_problem['choices']    # 선택지 리스트

print("문제:", question)
print("선택지:")
for idx, choice in enumerate(choices):
    print(f"{idx + 1}. {choice}")


문제: 윗글의 내용과 일치하지 않는 것은?
선택지:
1. 같은 책을 읽은 독자라도 서로 다른 의미를 구성할 수 있다.
2. 다른 독자와의 소통은 독자가 인식의 폭을 확장하도록 돕는다
3. 독자는 직접 경험해 보지 못했던 다양한 삶을 책의 필자를 매개로 접할 수 있다.
4. 독자의 배경지식, 관점, 읽기 환경, 과제는 독자의 의미 구성에 영향을 주는 독자 요인이다.
5. 독자는 책을 읽을 때 자신이 속한 사회나 시대의 영향을 받으며 필자와 간접적으로 대화한다


## [MY CODE] Open API를 호출하기 위해서, api key를 설정합니다.

In [5]:
import openai

# OpenAI API 키 설정
openai.api_key = ""


## [MY CODE]

evaluate_predictions 평가를 위한 함수를 정의합니다. 실제 answer와 gpt가 응답한 answer을 비교해서 score를 채점하는 코드입니다.

In [ ]:
def evaluate_predictions(data):
    """
    전체 문제를 대상으로 GPT-4의 예측 결과를 평가하는 함수.
    지문을 매번 제공하여 정확한 문맥을 모델에 전달.
    """
    total_score = 0  # 총 점수
    achieved_score = 0  # GPT-4가 획득한 점수
    curProblem = 0

    for i, item in enumerate(data):
        paragraph = item["paragraph"]  # 지문 가져오기

        print(f"지문 {i + 1}:")
        for problem in item["problems"]:
            # 문제별 데이터
            score = problem["score"]  # 문제의 가중치
            actual_answer = problem["answer"]  # 실제 정답

            # GPT-4 예측
            predicted_answer = prediction(problem, paragraph)

            # 숫자로 응답하지 않았을 경우 예외 처리
            if not predicted_answer.isdigit() or int(predicted_answer) not in range(1, 6):
                raise ValueError(f"Unexpected response: {predicted_answer}")

            # 결과 출력
            curProblem += 1
            print(f"문제 {curProblem}:")
            print(f"예측 답안: {predicted_answer}, 실제 정답: {actual_answer}")
            
            # 점수 계산
            total_score += score
            if predicted_answer == str(actual_answer):  # 정답 비교
                achieved_score += score

    # 최종 결과 출력
    print("\n최종 결과 요약:")
    print(f"총 점수: {achieved_score}/{total_score}")
    print(f"정확도: {achieved_score / total_score:.2%}")


## [MY CODE]
max token값을 5로 하고 temperature = 0으로 두어서 창의적인 응답을 제한하고 주어진 응답만 할수 있도록 유도합니다.
"role": "system", "content": "당신은 수능 국어 문제를 푸는 전문 AI입니다." 을 두어 초기적인 프롬프트를 지정합니다.

In [35]:
import time

def safe_api_call(prompt, max_tokens, retries=3):
    """
    GPT-4 API 호출을 안전하게 처리하는 함수.
    RateLimitError 발생 시 재시도 로직 포함.
    """
    for _ in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "당신은 수능 국어 문제를 푸는 전문 AI입니다."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=5,  # 짧은 응답을 받기 위해 제한
                temperature=0
            )
            return response
        except openai.error.RateLimitError:
            print("Rate limit exceeded. Retrying...")
            time.sleep(60)  # 1분 후 재시도
    raise Exception("Rate limit exceeded after multiple retries.")

## [MY CODE] 첫번째 프롬프트 실험 진행
첫번째 프롬프트를 지정합니다. 프롬프트 전략을 찾아보았고 적용되는 프롬프트 전략들만 추려내어 작성합니다. 그 내용은 다음과 같습니다.

1. **본론 바로 들어가기**
    - `당신은 대한민국 수능 국어 문제를 해결하는 전문 AI입니다.`: 역할 바로 명시.
    - `당신의 임무는 지문과 문제를 분석한 후 가장 적절한 선택지를 고르는 것입니다.`: 핵심 작업 설명.

2. **청중 설정**
    - `당신은 대한민국 수능 국어 문제를 해결하는 전문 AI입니다.`: 특정 청중과 상황 설정.

3. **작업 세분화**
    - `1. 문제를 논리적으로 분석합니다.`
    - `2. 단계적으로 생각하며 지문에서 중요한 단서를 찾아냅니다.`
    - `3. 선택지 중 하나를 정확히 선택합니다.`: 문제 풀이 단계를 나눔.

4. **긍정 지시문 사용**
    - `가장 적절한 선택지를 고르는 것입니다.`
    - `정확히 선택합니다.`: 긍정적인 표현으로 작업 지시.

5. **예시 제공**
    - `답변은 반드시 숫자만 작성하세요. (예: 1, 2, 3, 4, 5 중 하나)`: 답변 형식에 대한 명확한 예시 제공.

6. **프롬프트 형식 설정**
    - `### 역할 부여 ###`, `### 작업 지침 ###`, `### 지문 ###`: 명확한 섹션 구분으로 구조화.

7. **임무 부여**
    - `당신의 임무는 지문과 문제를 분석한 후 가장 적절한 선택지를 고르는 것입니다.`: 작업 명확화.

8. **단계별로 생각하기**
    - `2. 단계적으로 생각하며 지문에서 중요한 단서를 찾아냅니다.`: 순차적 사고 유도.

9. **역할 부여**
    - `당신은 대한민국 수능 국어 문제를 해결하는 전문 AI입니다.`: 전문성과 역할 강조.

10. **구분 기호 사용**
    - `### 역할 부여 ###`, `### 지문 ###`, `### 문제 ###`: 구조를 명확히 하여 작업 구분.

11. **키워드 반복**
    - `문제`, `선택지`, `정답`: 반복되는 핵심 단어로 초점 강화.

12. **연쇄 사고 논리**
    - `단계적으로 생각하며 지문에서 중요한 단서를 찾아냅니다.`: 연쇄적 사고를 강조.

13. **출력 문구 지정**
    - `답변은 반드시 숫자만 작성하세요. (예: 1, 2, 3, 4, 5 중 하나)`: 명확한 출력 요구.

14. **모든 정보 포함**
    - `### 지문 ### {paragraph}`, `### 문제 ### {question_text}`, `### 선택지 ### {choices}`: 모든 관련 정보를 포함.

15. **제시어 기반 글쓰기**
    - `숫자만 작성하세요.`: 특정 답변 형식 유도.

16. **요구사항 명확히 제시**
    - "단계적으로 생각", "정확히 선택": 명확한 지시로 작업 요구사항 전달.

In [ ]:
def prediction(problem, paragraph):
    """
    문제를 받아 지문과 함께 GPT-4 모델로 정답을 예측하는 함수.
    - problem: 문제 JSON 데이터
    - paragraph: 지문 텍스트
    """
    # 문제 내용 및 선택지 파싱
    question_text = problem["question"]
    choices = problem["choices"]
    
    # 프롬프트 생성
    prompt = f"""
    ### 역할 부여 ###
    당신은 대한민국 수능 국어 문제를 해결하는 전문 AI입니다.
    당신의 임무는 지문과 문제를 분석한 후 가장 적절한 선택지를 고르는 것입니다.

    ### 작업 지침 ###
    1. 문제를 논리적으로 분석합니다.
    2. 단계적으로 생각하며 지문에서 중요한 단서를 찾아냅니다.
    3. 선택지 중 하나를 정확히 선택합니다.
    4. 반드시 숫자로만 응답하세요. (예: 1, 2, 3, 4, 5)

    ### 지문 ###
    {paragraph}
    
    ### 문제 ###
    {question_text}

    ### 선택지 ###
    선택지:
    1. {choices[0]}
    2. {choices[1]}
    3. {choices[2]}
    4. {choices[3]}
    5. {choices[4]}

    ### 답변 형식 ###
    문제를 분석한 결과, 최종 답변은 반드시 숫자로만 작성합니다. (예: 1, 2, 3, 4, 5)
    """

    # GPT-4 모델 호출
    response = safe_api_call(prompt, max_tokens=5)
    return response.choices[0].message["content"].strip()

chat gpt 4o api를 호출하여 결과를 측정합니다. 해당 결과 68점을 얻을수 있었습니다.

In [ ]:
# 전체 데이터를 대상으로 평가 실행
evaluate_predictions(data)

지문 1:
문제 1:
예측 답안: 5, 실제 정답: 4
문제 2:
예측 답안: 3, 실제 정답: 5
문제 3:
예측 답안: 1, 실제 정답: 1
지문 2:
문제 4:
예측 답안: 4, 실제 정답: 4
문제 5:
예측 답안: 5, 실제 정답: 5
문제 6:
예측 답안: 3, 실제 정답: 3
문제 7:
예측 답안: 2, 실제 정답: 2
문제 8:
예측 답안: 3, 실제 정답: 5
문제 9:
예측 답안: 2, 실제 정답: 2
지문 3:
문제 10:
예측 답안: 1, 실제 정답: 4
문제 11:
예측 답안: 5, 실제 정답: 5
문제 12:
예측 답안: 3, 실제 정답: 2
문제 13:
예측 답안: 1, 실제 정답: 5
지문 4:
문제 14:
예측 답안: 3, 실제 정답: 3
문제 15:
예측 답안: 3, 실제 정답: 4
문제 16:
예측 답안: 4, 실제 정답: 4
문제 17:
예측 답안: 1, 실제 정답: 1
지문 5:
문제 18:
예측 답안: 4, 실제 정답: 4
문제 19:
예측 답안: 3, 실제 정답: 3
문제 20:
예측 답안: 3, 실제 정답: 3
문제 21:
예측 답안: 5, 실제 정답: 5
지문 6:
문제 22:
예측 답안: 1, 실제 정답: 1
문제 23:
예측 답안: 5, 실제 정답: 5
Rate limit exceeded. Retrying...
문제 24:
예측 답안: 3, 실제 정답: 3
문제 25:
예측 답안: 3, 실제 정답: 3
문제 26:
예측 답안: 4, 실제 정답: 4
지문 7:
문제 27:
예측 답안: 1, 실제 정답: 1
문제 28:
예측 답안: 5, 실제 정답: 5
문제 29:
예측 답안: 1, 실제 정답: 1
문제 30:
예측 답안: 2, 실제 정답: 2
지문 8:
문제 31:
예측 답안: 1, 실제 정답: 1
문제 32:
예측 답안: 4, 실제 정답: 4
문제 33:
예측 답안: 2, 실제 정답: 2
문제 34:
예측 답안: 1, 실제 정답: 3
지문 9:
문제 35:
예측 답안: 5, 실제 정답: 1
문제 36:
예측 답안

## [MY CODE] 두번째 프롬프트 실험 진행

프롬프트는 모델이나 모델의 복잡도에 따라서, 오히려 안좋아지는 경우도 발생한다는 정보를 접했습니다.
따라서, 기존 채점을 위한 출력을 제외하고 기존의 프롬프트들을 제거한 프롬프트를 작성했습니다.

In [43]:
import time

def safe_api_call(prompt, max_tokens, retries=3):
    """
    GPT-4 API 호출을 안전하게 처리하는 함수.
    RateLimitError 발생 시 재시도 로직 포함.
    """
    for _ in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "당신은 수능 국어 문제를 푸는 전문 AI입니다."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=5,  # 짧은 응답을 받기 위해 제한
                temperature=0
            )
            return response
        except openai.error.RateLimitError:
            print("Rate limit exceeded. Retrying...")
            time.sleep(60)  # 1분 후 재시도
    raise Exception("Rate limit exceeded after multiple retries.")

def prediction(problem, paragraph):
    """
    문제를 받아 지문과 함께 GPT-4 모델로 정답을 예측하는 함수.
    - problem: 문제 JSON 데이터
    - paragraph: 지문 텍스트
    """
    # 문제 내용 및 선택지 파싱
    question_text = problem["question"]
    choices = problem["choices"]
    
    # 프롬프트 생성
    prompt = f"""
    ### 지문 ###
    {paragraph}
    
    ### 문제 ###
    {question_text}

    ### 선택지 ###
    선택지:
    1. {choices[0]}
    2. {choices[1]}
    3. {choices[2]}
    4. {choices[3]}
    5. {choices[4]}

    ### 답변 형식 ###
    반드시 숫자로만 응답하세요. (예: 1, 2, 3, 4, 5)
    """

    # GPT-4 모델 호출
    response = safe_api_call(prompt, max_tokens=5)
    return response.choices[0].message["content"].strip()

이번에는 오히려 전보다 높은 73점의 점수를 얻을 걸 확인할수 있었습니다.

In [ ]:
# 전체 데이터를 대상으로 평가 실행
evaluate_predictions(data)

지문 1:
문제 1:
예측 답안: 4, 실제 정답: 4
문제 2:
예측 답안: 3, 실제 정답: 5
문제 3:
예측 답안: 1, 실제 정답: 1
지문 2:
문제 4:
예측 답안: 4, 실제 정답: 4
문제 5:
예측 답안: 5, 실제 정답: 5
문제 6:
예측 답안: 3, 실제 정답: 3
문제 7:
예측 답안: 2, 실제 정답: 2
문제 8:
예측 답안: 3, 실제 정답: 5
문제 9:
예측 답안: 2, 실제 정답: 2
지문 3:
문제 10:
예측 답안: 4, 실제 정답: 4
문제 11:
예측 답안: 5, 실제 정답: 5
문제 12:
예측 답안: 3, 실제 정답: 2
문제 13:
예측 답안: 1, 실제 정답: 5
지문 4:
문제 14:
예측 답안: 3, 실제 정답: 3
문제 15:
예측 답안: 3, 실제 정답: 4
문제 16:
예측 답안: 4, 실제 정답: 4
문제 17:
예측 답안: 1, 실제 정답: 1
지문 5:
문제 18:
예측 답안: 4, 실제 정답: 4
문제 19:
예측 답안: 3, 실제 정답: 3
문제 20:
예측 답안: 3, 실제 정답: 3
문제 21:
예측 답안: 5, 실제 정답: 5
지문 6:
문제 22:
예측 답안: 1, 실제 정답: 1
문제 23:
예측 답안: 5, 실제 정답: 5
문제 24:
예측 답안: 3, 실제 정답: 3
문제 25:
예측 답안: 3, 실제 정답: 3
문제 26:
예측 답안: 4, 실제 정답: 4
지문 7:
문제 27:
예측 답안: 1, 실제 정답: 1
Rate limit exceeded. Retrying...
문제 28:
예측 답안: 2, 실제 정답: 5
문제 29:
예측 답안: 1, 실제 정답: 1
문제 30:
예측 답안: 2, 실제 정답: 2
지문 8:
문제 31:
예측 답안: 1, 실제 정답: 1
문제 32:
예측 답안: 4, 실제 정답: 4
문제 33:
예측 답안: 2, 실제 정답: 2
문제 34:
예측 답안: 3, 실제 정답: 3
지문 9:
문제 35:
예측 답안: 3, 실제 정답: 1
문제 36:
예측 답안

## [MY CODE] 세번째 프롬프트 진행

위의 결과에서 응답형식을 수정해줍니다. 모델이 '숫자로만 입력합니다.'와 같은 강제적이고 특정 형식으로 해석해야 하는 문구 대신에<br/>
'최종 정답: (최종 정답)' 과 같이 단순하면서 더 이해할수 있는 형식으로 변환해줍니다. 이는 모델이 문제 해결에 더 집중할수 있게 의도했습니다.

In [ ]:
def prediction(problem, paragraph):
    """
    문제를 받아 지문과 함께 GPT-4 모델로 정답을 예측하는 함수.
    """
    question_text = problem["question"]
    choices = problem["choices"]

    # 프롬프트 생성
    prompt = f"""
    ### 지문 ###
    {paragraph}
    
    ### 문제 ###
    {question_text}

    ### 선택지 ###
    선택지:
    1. {choices[0]}
    2. {choices[1]}
    3. {choices[2]}
    4. {choices[3]}
    5. {choices[4]}

    다음의 형식을 따라 답변하세요.
    최종 정답: (최종 정답)  
    """

    # GPT-4 모델 호출
    response = safe_api_call(prompt, max_tokens=300)
    return response.choices[0].message["content"].strip()

실행 시켜보니 전보다 높은 79점을 얻는걸 확인할수 있습니다.


In [ ]:
evaluate_predictions(data)

지문 1:

문제 1:
예측 답안: 4, 실제 정답: 4

문제 2:
예측 답안: 4, 실제 정답: 5

문제 3:
예측 답안: 1, 실제 정답: 1
지문 2:

문제 4:
예측 답안: 4, 실제 정답: 4

문제 5:
예측 답안: 5, 실제 정답: 5

문제 6:
예측 답안: 3, 실제 정답: 3

문제 7:
예측 답안: 2, 실제 정답: 2

문제 8:
예측 답안: 3, 실제 정답: 5

문제 9:
예측 답안: 2, 실제 정답: 2
지문 3:

문제 10:
예측 답안: 4, 실제 정답: 4

문제 11:
예측 답안: 5, 실제 정답: 5

문제 12:
예측 답안: 3, 실제 정답: 2

문제 13:
예측 답안: 1, 실제 정답: 5
지문 4:

문제 14:
예측 답안: 3, 실제 정답: 3

문제 15:
예측 답안: 5, 실제 정답: 4

문제 16:
예측 답안: 5, 실제 정답: 4

문제 17:
예측 답안: 1, 실제 정답: 1
지문 5:

문제 18:
예측 답안: 4, 실제 정답: 4

문제 19:
예측 답안: 3, 실제 정답: 3

문제 20:
예측 답안: 3, 실제 정답: 3

문제 21:
예측 답안: 5, 실제 정답: 5
지문 6:

문제 22:
예측 답안: 1, 실제 정답: 1

문제 23:
예측 답안: 5, 실제 정답: 5

문제 24:
예측 답안: 3, 실제 정답: 3

문제 25:
예측 답안: 3, 실제 정답: 3
Rate limit exceeded. Retrying...

문제 26:
예측 답안: 4, 실제 정답: 4
지문 7:

문제 27:
예측 답안: 1, 실제 정답: 1

문제 28:
예측 답안: 5, 실제 정답: 5

문제 29:
예측 답안: 1, 실제 정답: 1

문제 30:
예측 답안: 2, 실제 정답: 2
지문 8:

문제 31:
예측 답안: 1, 실제 정답: 1

문제 32:
예측 답안: 4, 실제 정답: 4

문제 33:
예측 답안: 2, 실제 정답: 2

문제 34:
예측 답안: 3, 실제 정답: 3
지문 9:

문제 

## [MY CODE] 네번째 프롬프트 진행

위의 로그를 보면, '최종점수 : (최종점수)' 식으로 응답을 해야 했지만, 그렇게 응답하지 않아 즉 모델이 정해진 설정을 지키지 않아서 '예상치 못한 응답 형식입니다. 로그를 확인하세요.' 해당 오류에 걸려서 아예 틀려버린 문제도 존재했습니다.

모델이 정해진대로만 응답하게 수정을 해줍니다. 일단 프롬프트에 점수를 크게 삭감한다는 문구를 적용하여 정해진 대로만 응답하도록 수정합니다.

In [ ]:
def prediction(problem, paragraph):
    """
    문제를 받아 지문과 함께 GPT-4 모델로 정답을 예측하는 함수.
    """
    question_text = problem["question"]
    choices = problem["choices"]

    # 프롬프트 생성
    prompt = f"""
    ### 지문 ###
    {paragraph}
    
    ### 문제 ###
    {question_text}

    ### 선택지 ###
    선택지:
    1. {choices[0]}
    2. {choices[1]}
    3. {choices[2]}
    4. {choices[3]}
    5. {choices[4]}

    다음의 형식을 따라 답변하세요.
    최종 정답: (최종 정답)  
      
    형식을 어길 경우 점수가 대폭 감점됩니다. 따라서 무조건 반드시 지키세요.
    """

    # GPT-4 모델 호출
    response = safe_api_call(prompt, max_tokens=300)
    return response.choices[0].message["content"].strip()

결과를 채점해본결과, 이전보다 높은 81점을 얻고 실제로 형식 이외의 답변을 하지 않을 걸 확인할수 있었습니다.

In [78]:
evaluate_predictions(data)

지문 1:

문제 1:
예측 답안: 4, 실제 정답: 4

문제 2:
예측 답안: 4, 실제 정답: 5

문제 3:
예측 답안: 1, 실제 정답: 1
지문 2:

문제 4:
예측 답안: 4, 실제 정답: 4

문제 5:
예측 답안: 5, 실제 정답: 5

문제 6:
예측 답안: 3, 실제 정답: 3

문제 7:
예측 답안: 2, 실제 정답: 2

문제 8:
예측 답안: 3, 실제 정답: 5

문제 9:
예측 답안: 2, 실제 정답: 2
지문 3:

문제 10:
예측 답안: 4, 실제 정답: 4

문제 11:
예측 답안: 5, 실제 정답: 5

문제 12:
예측 답안: 3, 실제 정답: 2

문제 13:
예측 답안: 1, 실제 정답: 5
지문 4:

문제 14:
예측 답안: 3, 실제 정답: 3

문제 15:
예측 답안: 5, 실제 정답: 4

문제 16:
예측 답안: 5, 실제 정답: 4

문제 17:
예측 답안: 1, 실제 정답: 1
지문 5:

문제 18:
예측 답안: 4, 실제 정답: 4

문제 19:
예측 답안: 3, 실제 정답: 3

문제 20:
예측 답안: 3, 실제 정답: 3

문제 21:
예측 답안: 5, 실제 정답: 5
지문 6:

문제 22:
예측 답안: 1, 실제 정답: 1

문제 23:
예측 답안: 5, 실제 정답: 5

문제 24:
예측 답안: 3, 실제 정답: 3

문제 25:
예측 답안: 3, 실제 정답: 3

문제 26:
예측 답안: 4, 실제 정답: 4
지문 7:

문제 27:
예측 답안: 1, 실제 정답: 1
Rate limit exceeded. Retrying...

문제 28:
예측 답안: 5, 실제 정답: 5

문제 29:
예측 답안: 1, 실제 정답: 1

문제 30:
예측 답안: 2, 실제 정답: 2
지문 8:

문제 31:
예측 답안: 1, 실제 정답: 1

문제 32:
예측 답안: 4, 실제 정답: 4

문제 33:
예측 답안: 2, 실제 정답: 2

문제 34:
예측 답안: 3, 실제 정답: 3
지문 9:

문제 

## [MY CODE] 5번째 프롬프트 진행

chat gpt api를 호출하는 옵션중에, temperature 옵션이 존재합니다. 해당 항목은 진행중에 확률이 높은 여러개의 토큰중 창의적인 선택을 하도록 유도하는 옵션입니다. 하지만, 해당 국어 문제를 푸는데 있어서 해당 옵션은 정해진 틀을 벗어나도록 유도할수 있어서 마이너스 요소가 될수 있다고 생각했습니다. 해당 옵션을 0으로 하고 다시 실행시킵니다.

In [ ]:
for _ in range(retries):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "당신은 수능 국어 문제를 푸는 전문 AI입니다."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=5,
            # 해당 옵션을 0로 설정한다.
            temperature=0
        )
        return response
    except openai.error.RateLimitError:
        print("Rate limit exceeded. Retrying...")
        time.sleep(60)  # 1분 후 재시도
raise Exception("Rate limit exceeded after multiple retries.")

채점하고 실행해본결과 이전 81점보다 높은 83점의 점수를 얻은걸 확인할수 있습니다.

In [79]:
evaluate_predictions(data)

지문 1:

문제 1:
예측 답안: 4, 실제 정답: 4

문제 2:
예측 답안: 4, 실제 정답: 5

문제 3:
예측 답안: 1, 실제 정답: 1
지문 2:

문제 4:
예측 답안: 4, 실제 정답: 4

문제 5:
예측 답안: 5, 실제 정답: 5

문제 6:
예측 답안: 3, 실제 정답: 3

문제 7:
예측 답안: 2, 실제 정답: 2

문제 8:
예측 답안: 3, 실제 정답: 5

문제 9:
예측 답안: 2, 실제 정답: 2
지문 3:

문제 10:
예측 답안: 4, 실제 정답: 4

문제 11:
예측 답안: 5, 실제 정답: 5

문제 12:
예측 답안: 3, 실제 정답: 2

문제 13:
예측 답안: 1, 실제 정답: 5
지문 4:

문제 14:
예측 답안: 3, 실제 정답: 3

문제 15:
예측 답안: 5, 실제 정답: 4

문제 16:
예측 답안: 5, 실제 정답: 4

문제 17:
예측 답안: 1, 실제 정답: 1
지문 5:

문제 18:
예측 답안: 4, 실제 정답: 4

문제 19:
예측 답안: 3, 실제 정답: 3

문제 20:
예측 답안: 3, 실제 정답: 3

문제 21:
예측 답안: 5, 실제 정답: 5
지문 6:

문제 22:
예측 답안: 1, 실제 정답: 1

문제 23:
예측 답안: 5, 실제 정답: 5

문제 24:
예측 답안: 3, 실제 정답: 3

문제 25:
예측 답안: 3, 실제 정답: 3

문제 26:
예측 답안: 4, 실제 정답: 4
지문 7:

문제 27:
예측 답안: 1, 실제 정답: 1

문제 28:
예측 답안: 5, 실제 정답: 5

문제 29:
예측 답안: 1, 실제 정답: 1
Rate limit exceeded. Retrying...

문제 30:
예측 답안: 2, 실제 정답: 2
지문 8:

문제 31:
예측 답안: 1, 실제 정답: 1

문제 32:
예측 답안: 4, 실제 정답: 4

문제 33:
예측 답안: 2, 실제 정답: 2

문제 34:
예측 답안: 3, 실제 정답: 3
지문 9:

문제 

## [MY CODE] 결론

1. **모델의 복잡도에 따라서, 프롬프트들은 달라질수 있다.**
    - 여러 프롬프트 테크닉을 넣는다고 해서 무조건 좋은 성능을 보장하지는 않다.

2. **최대한 형식적이고 단순하게**
    - `최종점수 : (최종점수)`처럼 내가 출력을 하려는 문구가 있다면 최대한 구체적으로 넣어주자. '강제', '의도'처럼 단어를 통해서 유도하지 말고 아예 원하는 형식을 전달해주는게 좋다.

3. **유도**
    - 효과적으로 형식을 유도하는걸 벌점을 넣거나 페널티를 준다고 의도하는것이다. 해당 문구를 프롬프트에 넣는것이 오히려 효율적일수 있다.

4. **창의적 ? 형식적 ?**
    - 항상 목적에 맞게 모델을 선택해야 한다. 창의적인 작업에는 temperature 설정을 줘서 창의적인 결과를 유도할수 있고 아니라면 temperature을 낮춰서 정해진 결과만을 하도록 유도할수 있다.